In [4]:
!pip install -U tfx

     |████████████████████████████████| 2.5 MB 5.2 MB/s 
     |████████████████████████████████| 133 kB 19.1 MB/s 
     |████████████████████████████████| 10.2 MB 45.2 MB/s 
     |████████████████████████████████| 23.6 MB 1.4 MB/s 
     |████████████████████████████████| 49 kB 5.8 MB/s 
     |████████████████████████████████| 147 kB 59.6 MB/s 
     |████████████████████████████████| 1.3 MB 43.0 MB/s 
     |████████████████████████████████| 6.6 MB 33.9 MB/s 
     |████████████████████████████████| 40 kB 6.2 MB/s 
     |████████████████████████████████| 135 kB 47.0 MB/s 
     |████████████████████████████████| 206 kB 53.2 MB/s 
     |████████████████████████████████| 1.8 MB 52.5 MB/s 
     |████████████████████████████████| 1.7 MB 30.0 MB/s 
     |████████████████████████████████| 433 kB 52.7 MB/s 
     |████████████████████████████████| 19.2 MB 1.2 MB/s 
     |████████████████████████████████| 1.4 MB 47.8 MB/s 
     |████████████████████████████████| 1.8 MB 41.5 MB/s 
     |████████████

In [16]:
import os
import tempfile
import urllib
import tensorflow_data_validation as tfdv
import pandas as pd

In [45]:
_data_root = tempfile.mkdtemp(prefix='tfx-data')
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
_data_filepath = os.path.join(_data_root, "data.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

# 데이터 통계 요약 정보 생성
stats = tfdv.generate_statistics_from_csv(
    data_location=_data_filepath,
    delimiter=','
)

# print(stats)
tfdv.visualize_statistics(stats)

In [34]:
# 스키마 생성
schema = tfdv.infer_schema(stats)
# print(schema)
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'pickup_community_area',INT,optional,single,-
'fare',FLOAT,required,,-
'trip_start_month',INT,required,,-
'trip_start_hour',INT,required,,-
'trip_start_day',INT,required,,-
'trip_start_timestamp',INT,required,,-
'pickup_latitude',FLOAT,optional,single,-
'pickup_longitude',FLOAT,optional,single,-
'dropoff_latitude',FLOAT,optional,single,-


,Values
Domain,
'payment_type',"'Cash', 'Credit Card', 'Dispute', 'No Charge', 'Pcard', 'Prcard', 'Unknown'"
'company',"'0118 - 42111 Godfrey S.Awir', '0694 - 59280 Chinesco Trans Inc', '1085 - 72312 N and W Cab Co', '2092 - 61288 Sbeih company', '2192 - 73487 Zeymane Corp', '2192 - Zeymane Corp', '2733 - 74600 Benny Jona', '2809 - 95474 C & D Cab Co Inc.', '2823 - 73307 Seung Lee', '3011 - 66308 JBL Cab Inc.', '3094 - 24059 G.L.B. Cab Co', '3152 - 97284 Crystal Abernathy', '3201 - C&D Cab Co Inc', '3201 - CID Cab Co Inc', '3253 - 91138 Gaither Cab Co.', '3319 - CD Cab Co', '3385 - 23210 Eman Cab', '3385 - Eman Cab', '3623 - 72222 Arrington Enterprises', '3897 - 57856 Ilie Malec', '3897 - Ilie Malec', '4053 - 40193 Adwar H. Nikola', '4053 - Adwar H. Nikola', '4197 - 41842 Royal Star', '4197 - Royal Star', '4615 - 83503 Tyrone Henderson', '4615 - Tyrone Henderson', '4623 - Jay Kim', '5006 - 39261 Salifu Bawa', '5006 - Salifu Bawa', '5074 - 54002 Ahzmi Inc', '5074 - Ahzmi Inc', '5129 - 87128', '5129 - 98755 Mengisti Taxi', '5129 - Mengisti Taxi', '5724 - KYVI Cab Inc', '585 - 88805 Valley Cab Co', '585 - Valley Cab Co', '5864 - 73614 Thomas Owusu', '5864 - Thomas Owusu', '5874 - 73628 Sergey Cab Corp.', '5874 - Sergey Cab Corp.', '5997 - 65283 AW Services Inc.', '5997 - AW Services Inc.', '6057 - 24657 Richard Addo', '6488 - 83287 Zuha Taxi', '6574 - Babylon Express Inc.', '6742 - 83735 Tasha ride inc', '6743 - Luhak Corp', 'Blue Ribbon Taxi Association Inc.', 'C & D Cab Co Inc', 'Chicago Elite Cab Corp.', 'Chicago Elite Cab Corp. (Chicago Carriag', 'Chicago Medallion Leasing INC', 'Chicago Medallion Management', 'Choice Taxi Association', 'Dispatch Taxi Affiliation', 'KOAM Taxi Association', 'Northwest Management LLC', 'Taxi Affiliation Services', 'Top Cab Affiliation'"


In [35]:
# data set 비교
df = pd.read_csv(DATA_PATH)
print(df.shape)

df_train = df.iloc[:10000, :]
df_eval = df.iloc[10000:, :]

print(df_train.shape)
print(df_eval.shape)

train_stats = tfdv.generate_statistics_from_dataframe(df_train)
eval_stats = tfdv.generate_statistics_from_dataframe(df_eval)

tfdv.visualize_statistics(train_stats, eval_stats, lhs_name="TRAIN DATASET", rhs_name="EVAL DATASET")

(15002, 18)
(10000, 18)
(5002, 18)


In [39]:
# 이상치 탐지 시각화
schema = tfdv.infer_schema(train_stats)

anomalies = tfdv.validate_statistics(statistics=train_stats, schema=schema)

tfdv.display_anomalies(anomalies)

print(anomalies)

baseline {
  feature {
    name: "pickup_community_area"
    value_count {
      min: 1
      max: 1
    }
    type: FLOAT
    presence {
      min_count: 1
    }
  }
  feature {
    name: "fare"
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "trip_start_month"
    type: INT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "trip_start_hour"
    type: INT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "trip_start_day"
    type: INT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "trip_start_timestamp"
    type: INT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
   

In [40]:
from tensorflow_data_validation.utils.display_util import display_anomalies
# 스키마 업데이트

# 기존 스키마 저장
_schema_filepath = os.path.join(_data_root, "my_schema")
tfdv.write_schema_text(schema, _schema_filepath)

# 저장된 기존 스키마를 불러온다.
schema = tfdv.load_schema_text(_schema_filepath)

# dropoff_census_tract 피처를 90% 이상 포함되도록한다.
dropoff_census_tract_feature = tfdv.get_feature(schema, 'dropoff_census_tract')
dropoff_census_tract_feature.presence.min_fraction = 0.9

# payment_type 도메인 중 "Cash"를 삭제한다.
payment_type_domain = tfdv.get_domain(schema, 'payment_type')
payment_type_domain.value.remove("Cash")

# 변경된 스키마를 저장
tfdv.write_schema_text(schema, _schema_filepath)

# 변경된 스키마를 불러온다.
schema = tfdv.load_schema_text(_schema_filepath)

# 다시 확인
stats = tfdv.generate_statistics_from_csv(
    data_location=_data_filepath,
    delimiter=','
)

update = tfdv.validate_statistics(stats, schema)
tfdv.display_anomalies(update)

,Anomaly short description,Anomaly long description
Feature name,,
'company',Unexpected string values,"Examples contain values missing from the schema: 3897 - Ilie Malec (<1%), 5074 - Ahzmi Inc (<1%), 585 - Valley Cab Co (<1%), 5864 - Thomas Owusu (<1%), 5997 - AW Services Inc. (<1%)."
'pickup_census_tract',Expected data of type: FLOAT but got INT,
'pickup_community_area',Expected data of type: FLOAT but got INT,
'payment_type',Unexpected string values,Examples contain values missing from the schema: Cash (~66%).
'dropoff_census_tract',Column dropped,"The feature was present in fewer examples than expected: minimum fraction = 0.900000, actual = 0.717304"
'trip_seconds',Expected data of type: FLOAT but got INT,
'dropoff_community_area',Expected data of type: FLOAT but got INT,


In [51]:
# 머신러닝 파이프라인에 통합하기
from tfx import v1 as tfx
from tfx.components import CsvExampleGen
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import example_gen_pb2

# 학습, 평가, 테스트 데이터셋을 각각 6:2:2의 비율로 분할 / 분할 비율은 hash_buckets이 담당 / 기본 분할은 2:1 (학습, 평가)
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
                                                     example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=6),
                                                     example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2),
                                                     example_gen_pb2.SplitConfig.Split(name='test', hash_buckets=2),
    ])
)
context = InteractiveContext()

# 파이프라인 컴포넌트를 인스턴스화합니다.
example_gen = CsvExampleGen(input_base=_data_root, output_config=output)
# 대화식으로 컴포넌트를 실행합니다.
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [55]:
from tfx.components import StatisticsGen

statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)
context.show(statistics_gen.outputs['statistics'])

In [56]:
from tfx.components import SchemaGen

schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'],
                       infer_feature_shape=True)
context.run(schema_gen)
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'company',STRING,required,,'company'
'payment_type',STRING,required,,'payment_type'
'dropoff_census_tract',INT,required,,-
'dropoff_community_area',INT,required,,-
'dropoff_latitude',FLOAT,required,,-
'dropoff_longitude',FLOAT,required,,-
'fare',FLOAT,required,,-
'pickup_census_tract',INT,required,,-
'pickup_community_area',INT,required,,-


,Values
Domain,
'company',"'0118 - 42111 Godfrey S.Awir', '0694 - 59280 Chinesco Trans Inc', '1085 - 72312 N and W Cab Co', '2092 - 61288 Sbeih company', '2192 - 73487 Zeymane Corp', '2192 - Zeymane Corp', '2733 - 74600 Benny Jona', '2809 - 95474 C & D Cab Co Inc.', '2823 - 73307 Seung Lee', '3011 - 66308 JBL Cab Inc.', '3094 - 24059 G.L.B. Cab Co', '3152 - 97284 Crystal Abernathy', '3201 - C&D Cab Co Inc', '3201 - CID Cab Co Inc', '3253 - 91138 Gaither Cab Co.', '3319 - CD Cab Co', '3385 - Eman Cab', '3623 - 72222 Arrington Enterprises', '3897 - Ilie Malec', '4053 - Adwar H. Nikola', '4197 - 41842 Royal Star', '4197 - Royal Star', '4615 - 83503 Tyrone Henderson', '4615 - Tyrone Henderson', '5006 - 39261 Salifu Bawa', '5074 - 54002 Ahzmi Inc', '5074 - Ahzmi Inc', '5129 - 87128', '5129 - Mengisti Taxi', '5724 - KYVI Cab Inc', '5864 - 73614 Thomas Owusu', '5864 - Thomas Owusu', '5874 - 73628 Sergey Cab Corp.', '5997 - 65283 AW Services Inc.', '5997 - AW Services Inc.', '6057 - 24657 Richard Addo', '6488 - 83287 Zuha Taxi', '6574 - Babylon Express Inc.', 'Blue Ribbon Taxi Association Inc.', 'C & D Cab Co Inc', 'Chicago Elite Cab Corp.', 'Chicago Elite Cab Corp. (Chicago Carriag', 'Chicago Medallion Leasing INC', 'Chicago Medallion Management', 'Choice Taxi Association', 'Dispatch Taxi Affiliation', 'KOAM Taxi Association', 'Northwest Management LLC', 'Taxi Affiliation Services', 'Top Cab Affiliation', '3385 - 23210 Eman Cab', '3897 - 57856 Ilie Malec', '4623 - Jay Kim', '5006 - Salifu Bawa', '5129 - 98755 Mengisti Taxi', '585 - 88805 Valley Cab Co', '4053 - 40193 Adwar H. Nikola', '585 - Valley Cab Co', '5874 - Sergey Cab Corp.', '6742 - 83735 Tasha ride inc', '6743 - Luhak Corp'"
'payment_type',"'Cash', 'Credit Card', 'Dispute', 'No Charge', 'Pcard', 'Unknown', 'Prcard'"


In [54]:
# ExampleVlidator
from tfx.components import ExampleValidator

example_validator = ExampleValidator(statistics=statistics_gen.outputs['statistics'],
                                     schema=schema_gen.outputs['schema'])

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))